{{< include _webr-include.qmd >}}

# Types of optimization problems

There are two broad kinds of optimization problems that you are likely to encounter: those with continuous parameters, and those with discrete parameters.

## Continuous parameters

In the case of continuous parameters (like the gravity example before), the basic task is to where the derivative of loss with respect to the parameters is zero. Most (all?) algorithms will find a *local* minimum, with no guarantee that it finds the global minimum. There are two basic categories of optimizer for continuous parameters: *closed form* and *iterative*. A closed form optimizer is a special solution to a specific problem, and generally can't be applied to a different problem. So, most (all?) generic optimizers are iterative: they progress toward the solution in repeated steps until the steps get small enough that we say the algorithm has *converged*.

Next, we can break optimization algorithms into categories based on how much information we have about the derivatives of the function w.r.t. the parameters. The most common categories are to have:
0. No knowledge of the derivatives.
1. Known first derivative.
2. Known first and second derivatives.



Here is an example:

```{webr grad-descent-setup-grid, echo=FALSE}
source("https://raw.githubusercontent.com/wes-brooks/optimization/main/prepare_grid_plots.R")
```

### Newton-Raphson
This algorithm is for the case that you know first and second derivatives of the function you are optimizing. It is also important that the function be convex, or the solution may not be found.

Newton-Raphson optimization takes the Taylor expansion of the function to two terms, which means approximating the function as a parabola starting from some starting point. The minimum of a parabola is easy to find analytically (closed form), so we jump to that point and iterate the process. This makes more sense if you see it in action:

```{webr plot-newton-raphson-grid}
# first pass:
#drawIt(gplot_nr)

# second pass:
#res = iterate_nr(gplot_nr, xloc = 2.5)
#drawIt(res[["plot"]])

# third and sbsequent passes:
#res = iterate_nr(res[["plot"]], xloc = res[["x_new"]])
#drawIt(res[["plot"]])
```

Let's look now at an example of a nonconvex function:

```{webr plot-newton-raphson-grid-nonconvex}
# first pass:
#drawIt(gplot_nr_nonconvex)

# second pass:
#gp_nc = iterate_nr(gplot_nr_nonconvex, xloc = 1.3)
#drawIt(gp_nc[["plot"]])

# third and sbsequent passes:
#gp_nc = iterate_nr(gp_nc$plot, xloc = gp_nc$x_new)
#drawIt(gp_nc$plot)
```

Newton-Raphson is very fast because each step is closed-form. But because the Newton-Raphson algorithm requires evaluating to the second derivative, it can be very slow to optimize a high-dimensional problem (remember that you need to evaluate cross-derivatives of a vector function, so that's $p^2$ derivatives for $p$ parameters - but even wouldn't be so bad if you didn't then need to invert the second-derivative matrix. Truly nasty stuff. I've crashed clusters this way.)

### Gradient descent
When the second derivatives are unknown or impractical, gradient descent may work better. For gradient descent, you ony need the first derivative w.r.t. each parameter, then take a step in the direction of greatest improvement in the objective. This is simple but not so fast as Newton-Raphson because the size of each step must be tuned by a loop. Still, it is fast and simple and usually the best choice when you're in a situation of needing to call an optimizer (rather than a professionally engineered estimation function). There are many flavors of gradient descent to improve its speed and stability, but the only one worth mentioning here is conjugate gradient descent (commonly abbreviated CG), which includes some momentum from past steps in future steps. Again, an example may help illustrate the algorithm:


```{webr plot-grad-descent-grid}
# first pass:
#drawIt(gplot_gd)

# second pass:
#result_gd = iterate_gd(gplot_gd, gamma = 0.005, x_last = 2.5)
#drawIt(result_gd[["plot"]])

# third and sbsequent passes:
#result_gd = iterate_gd(result_gd[["plot"]], gamma = result_gd[["gamma"]], x_last = result_gd[["x_new"]])
#drawIt(result_gd[["plot"]])
```


Gradient descent is better-suited to optimizing our nonconvex function:
```{webr plot-grad-descent-grid-nc}
# first pass:
#drawIt(gplot_nonconvex)

# second pass:
#gp_gd_nc = iterate_gd_nonconvex(gplot_nonconvex, gamma = 0.015, x_last = 1.3)
#drawIt(gp_gd_nc[["plot"]])

# third and sbsequent passes:
#gp_gd_nc = iterate_gd_nonconvex(gp_gd_nc$plot, gamma = gp_gd_nc$gamma, x_last = gp_gd_nc$x_new)
#drawIt(gp_gd_nc$plot)
```


### Quick-and-dirty options
When your number of parameters is small and the data size is modest, you may not have to bother with calculating derivatives. The built-in general-purpose optimization functions in R and Python default to Nelder-Mead or BFGS algorithms, which essentially make each step by trying a bunch of candidates around the current location. They are "slow" in the computer, but fast for the scientist - and only the scientist gets paid for their time.

### Automatic differentiation
On the other hand, if your problem is very complicated, your easiest bet may be to use automatic differentiation and then conjugate gradient descent or Newton-Raphson. The widely used differentiation package is [`adcomp`, aka Template Model Builder (TMB)](https://github.com/kaskr/adcomp). You use it by writing your objective function as a C++ function template, and then the `adcomp` software hits everything with the chain rule as many ties as necessary to get derivatives. I love this but the barrier to entry is large. 

### Constrained optimization
Often, the reason to do custom optimization is because you need to constrain your solution, such as saying that all of the parameters should sum to one. Equality constraints are easy to implement via *Lagrange multipliers*. This is a fancy name for adding something like $\lambda (\sum_{i=1}^p \beta_i - 1)$ to your loss function. Since the minimum is found where the derivative of this term w.r.t. $\lambda$ is zero, the minimum is where $\sum_{i=1}^p \beta_i = 1$. Let's try it:

```{webr}
# import the Palmer penguins data
penguins = read.csv("https://raw.githubusercontent.com/allisonhorst/palmerpenguins/main/inst/extdata/penguins.csv")
head(penguins)

# This allows us to separate the masses by species.
species_indicator = model.matrix(~species+0, data=penguins) 
head(species_indicator)
```

Now we've looked at the data. Lets estimate the mass of each penguin species, with the constraint that Chinstrap and Adelie penguins have the same mass.

```{webr optimize-penguin-model}
# estimate mass of the three species
# with the constraint that Adelie and Chinstrap have the same mass
my_loss = function(par, mass, spmat) {
  sum( abs((mass - spmat %*% as.matrix(par[1:3]))), na.rm=TRUE) +
    par[4] * (par[1] - par[2])
}

# we have to minimized *squared* gradients now because Lagrange multipliers find
# a saddlepoint, which is only a local minimum.
opt_loss = function(par, mass, spp_mat) {
  par=par
  nd = numericDeriv(quote(my_loss(par, mass, spp_mat)), "par")
  sum(attr(nd, "gradient")^2)
}

# run the optimizer  
optim(par = c(3000, 3000, 5000, 0),
     fn = opt_loss,
     mass = penguins$body_mass_g,
     spp_mat = species_indicator)
```


## Discrete parameters

When parameters are discrete, the loss isn't continuous, so you can't minimize the function by finding where the derivatives w.r.t. the parameters are zero. For small problems 9like deciding which of three variables should be included in a regression model), it is easy enough to check all possible combinations and choose the best one. But to search all of the possible solutions to a large problem would be combinatorically impossible (e.g. the number of outcomes when shuffling a 52-card deck is on the order of the number of atoms in the universe). For problems like that, I'd recommend genetic algorithms. I'm not going to provide an example because they require packages outside of the base R or Python.

### Genetic algorithm
The gist of a genetic algorithm is that you define a "fitness" function (like our loss functions for continuous optiization) as well as a population of randomly selected combinations of parameters. The candidates in that population are then passed to the next generation with a probability proportional to their fitness and those that pass through are also randomly modified (aka "mutated") by a small amount. After any generations of this process, you may end up with a good solution to the optimization problem.